In [ ]:
# no warnings
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint


In [ ]:
df_application_record = pd.read_csv('/content/application_record.csv')
df_application_record.head(-5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438547,6839917,F,N,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,-10966,-2704,1,0,0,0,NaN,2.0
438548,6839936,M,Y,Y,1,135000.0,Working,Secondary / secondary special,Married,House / apartment,-12569,-2095,1,0,0,0,Laborers,3.0
438549,6840098,F,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438550,6840100,F,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0


In [ ]:
df_credit_record = pd.read_csv('/content/credit_record.csv')
df_credit_record.head(-5)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048565,5150487,-20,C
1048566,5150487,-21,C
1048567,5150487,-22,C
1048568,5150487,-23,C


In [ ]:
corresponding_ids = all('ID' == 'ID' for ID, ID in zip(df_application_record,df_credit_record))

if corresponding_ids:
    print('T')
else: print('F')

T


In [ ]:
df = pd.merge(df_application_record, df_credit_record, on = 'ID', how = 'left')

df.head(-20)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0.0,C
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-1.0,C
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-2.0,C
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-3.0,C
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179790,6836995,F,N,N,0,360000.0,Commercial associate,Higher education,Separated,House / apartment,-16374,-606,1,0,0,0,Managers,1.0,NaN,NaN
1179791,6837198,F,N,N,0,211500.0,Pensioner,Higher education,Separated,House / apartment,-20490,365243,1,0,0,0,NaN,1.0,NaN,NaN
1179792,6837235,F,N,Y,0,135000.0,Working,Secondary / secondary special,Married,House / apartment,-13589,-1093,1,0,0,0,Laborers,2.0,NaN,NaN
1179793,6837236,F,N,Y,0,135000.0,Working,Secondary / secondary special,Married,House / apartment,-13589,-1093,1,0,0,0,Laborers,2.0,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1179815 entries, 0 to 1179814
Data columns (total 20 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   ID                   1179815 non-null  int64  
 1   CODE_GENDER          1179815 non-null  object 
 2   FLAG_OWN_CAR         1179815 non-null  object 
 3   FLAG_OWN_REALTY      1179815 non-null  object 
 4   CNT_CHILDREN         1179815 non-null  int64  
 5   AMT_INCOME_TOTAL     1179815 non-null  float64
 6   NAME_INCOME_TYPE     1179815 non-null  object 
 7   NAME_EDUCATION_TYPE  1179815 non-null  object 
 8   NAME_FAMILY_STATUS   1179815 non-null  object 
 9   NAME_HOUSING_TYPE    1179815 non-null  object 
 10  DAYS_BIRTH           1179815 non-null  int64  
 11  DAYS_EMPLOYED        1179815 non-null  int64  
 12  FLAG_MOBIL           1179815 non-null  int64  
 13  FLAG_WORK_PHONE      1179815 non-null  int64  
 14  FLAG_PHONE           1179815 non-null  int64  
 15

In [ ]:
df.isnull().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        362928
CNT_FAM_MEMBERS             0
MONTHS_BALANCE         402100
STATUS                 402100
dtype: int64

In [ ]:
le = LabelEncoder()

df['CODE_GENDER'] = le.fit_transform(df['CODE_GENDER'])
df['FLAG_OWN_CAR'] = le.fit_transform(df['FLAG_OWN_CAR'])
df['FLAG_OWN_REALTY'] = le.fit_transform(df['FLAG_OWN_REALTY'])
df['CNT_CHILDREN'] = le.fit_transform(df['CNT_CHILDREN'])
df['NAME_INCOME_TYPE'] = le.fit_transform(df['NAME_INCOME_TYPE'])
df['NAME_EDUCATION_TYPE'] = le.fit_transform(df['NAME_EDUCATION_TYPE'])
df['NAME_FAMILY_STATUS'] = le.fit_transform(df['NAME_FAMILY_STATUS'])
df['NAME_HOUSING_TYPE'] = le.fit_transform(df['NAME_HOUSING_TYPE'])
df['OCCUPATION_TYPE'] = le.fit_transform(df['OCCUPATION_TYPE'])
df['STATUS'] = le.fit_transform(df['STATUS'])
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,18,2.0,0.0,6
1,5008804,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,18,2.0,-1.0,6
2,5008804,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,18,2.0,-2.0,6
3,5008804,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,18,2.0,-3.0,6
4,5008804,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,18,2.0,-4.0,6


In [ ]:
quantitative_col = ['AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'MONTHS_BALANCE']
df_quan = df[quantitative_col]
ss = StandardScaler()
df[quantitative_col] = ss.fit_transform(df_quan)
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,1,1,1,0,2.282274,4,1,0,4,0.98679,-0.461094,1,1,0,0,18,2.0,1.375748,6
1,5008804,1,1,1,0,2.282274,4,1,0,4,0.98679,-0.461094,1,1,0,0,18,2.0,1.304737,6
2,5008804,1,1,1,0,2.282274,4,1,0,4,0.98679,-0.461094,1,1,0,0,18,2.0,1.233725,6
3,5008804,1,1,1,0,2.282274,4,1,0,4,0.98679,-0.461094,1,1,0,0,18,2.0,1.162714,6
4,5008804,1,1,1,0,2.282274,4,1,0,4,0.98679,-0.461094,1,1,0,0,18,2.0,1.091702,6


In [ ]:
df_valori, df_senza = df.dropna(subset = ['MONTHS_BALANCE']), df[df['MONTHS_BALANCE'].isna()]
model_LG_Nan = LinearRegression()
model_LG_Nan.fit(df_valori[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'STATUS']],\
             df_valori['MONTHS_BALANCE'])
prev_LG_Nan = model_LG_Nan.predict(df_senza[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'STATUS']])
df.loc[df['MONTHS_BALANCE'].isna(), 'MONTHS_BALANCE'] = prev_LG_Nan

In [ ]:
df.isnull().sum()

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
MONTHS_BALANCE         0
STATUS                 0
dtype: int64

In [ ]:
X = df.drop('STATUS', axis = 1)
y = df['STATUS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)
X.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE
0,5008804,1,1,1,0,2.282274,4,1,0,4,0.98679,-0.461094,1,1,0,0,18,2.0,1.375748
1,5008804,1,1,1,0,2.282274,4,1,0,4,0.98679,-0.461094,1,1,0,0,18,2.0,1.304737
2,5008804,1,1,1,0,2.282274,4,1,0,4,0.98679,-0.461094,1,1,0,0,18,2.0,1.233725
3,5008804,1,1,1,0,2.282274,4,1,0,4,0.98679,-0.461094,1,1,0,0,18,2.0,1.162714
4,5008804,1,1,1,0,2.282274,4,1,0,4,0.98679,-0.461094,1,1,0,0,18,2.0,1.091702


**MODELLO: DECISION TREE**

In [ ]:
model_DTC = DecisionTreeClassifier()
param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(1, 50),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2', None]
}

random_search = RandomizedSearchCV(estimator=model_DTC, param_distributions=param_dist, cv=5, n_iter = 100, verbose = 1, n_jobs = -1, scoring='accuracy')

random_search.fit(X_train, y_train)


print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Parameters: {'criterion': 'entropy', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 7}
Best Score: 0.8655623766452347


In [ ]:
model_DTC= DecisionTreeClassifier(
    criterion='entropy',
    max_depth=35,
    min_samples_split=7,
    min_samples_leaf=2,
    max_features=None,
    random_state=0,
    class_weight = 'balanced'
)
model_DTC.fit(X_train, y_train)
y_pred = model_DTC.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Accuracy: 0.8909632852561838
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.79      0.81     87291
           1       0.18      0.38      0.24      2631
           2       0.07      0.20      0.11       225
           3       0.02      0.04      0.02        95
           4       0.02      0.07      0.03        58
           5       0.70      0.85      0.77       442
           6       0.93      0.90      0.92     99037
           7       0.75      0.81      0.78     43545
           8       1.00      1.00      1.00    120621

    accuracy                           0.89    353945
   macro avg       0.50      0.56      0.52    353945
weighted avg       0.90      0.89      0.89    353945



In [ ]:
conf_matrix_DTC = confusion_matrix(y_test, y_pred)
print("Matrice di Confusione:")
print(conf_matrix_DTC)

Matrice di Confusione:
[[ 73783    735     39      4      2     15   6913   5800      0]
 [  1767    479     28      6      0      2    252     97      0]
 [   107     54     16      6      4      6     20     12      0]
 [    39      9     21      1      4      7      8      6      0]
 [    20      1      3      5      2     14     11      2      0]
 [    43      8     14      2      8    336     12     19      0]
 [  7945    196     16      2      5     31  89889    953      0]
 [  9561    103      9      0      3     15   1982  31872      0]
 [     1      0      0      0      0      0      3      0 120617]]


In [ ]:
train_accuracy = model_DTC.score(X_train, y_train)
test_accuracy = model_DTC.score(X_test, y_test)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Training Accuracy: 0.9500587259496046
Test Accuracy: 0.8956052494031559


In [ ]:
importances = model_DTC.feature_importances_

for feature, importance in zip(X.columns, importances):
    print(f"{feature}: {importance}")

ID: 0.7256656423372396
CODE_GENDER: 0.0054604027459721005
FLAG_OWN_CAR: 0.00605115466396239
FLAG_OWN_REALTY: 0.0056192219207889015
CNT_CHILDREN: 0.004592953307847459
AMT_INCOME_TOTAL: 0.02984000062633178
NAME_INCOME_TYPE: 0.007875917227912872
NAME_EDUCATION_TYPE: 0.006889857297772888
NAME_FAMILY_STATUS: 0.009859011653682637
NAME_HOUSING_TYPE: 0.004317225001543341
DAYS_BIRTH: 0.049567058225412186
DAYS_EMPLOYED: 0.03542438344681941
FLAG_MOBIL: 0.0
FLAG_WORK_PHONE: 0.004101647316400273
FLAG_PHONE: 0.005507969515638179
FLAG_EMAIL: 0.0029194726350041606
OCCUPATION_TYPE: 0.018973831008567994
CNT_FAM_MEMBERS: 0.007055762395391746
MONTHS_BALANCE: 0.07027848867371221


**MODELLO: RANDOM FOREST**

In [ ]:
model_RF = RandomForestClassifier(
    n_estimators=100,
    max_depth=27,
    min_samples_split=2,
    min_samples_leaf=1)


model_RF.fit(X_train, y_train)
predictions_rf = model_RF.predict(X_test)
accuracy_model_rf = accuracy_score(y_test, predictions_rf)
report = classification_report(y_test, predictions_rf)
print(f'Accuracy: {accuracy_model_rf}')
print(f'Classification report: {report}')

Accuracy: 0.9047253104295865
Classification report:               precision    recall  f1-score   support

           0       0.83      0.84      0.84     87291
           1       0.38      0.19      0.25      2631
           2       0.20      0.11      0.14       225
           3       0.09      0.03      0.05        95
           4       0.09      0.07      0.08        58
           5       0.87      0.77      0.81       442
           6       0.88      0.95      0.91     99037
           7       0.85      0.73      0.78     43545
           8       1.00      1.00      1.00    120621

    accuracy                           0.90    353945
   macro avg       0.58      0.52      0.54    353945
weighted avg       0.90      0.90      0.90    353945



In [ ]:
conf_matrix_rf = confusion_matrix(y_test, predictions_rf)
print("Matrice di Confusione:")
print(conf_matrix_rf)

Matrice di Confusione:
[[ 73161    599     27      2      1      8   8653   4840      0]
 [  1662    491     42      1      0      1    355     79      0]
 [    65     74     25     12      3      2     30     14      0]
 [    19     10     20      3     18      3     17      5      0]
 [    14      2      1     12      4     10     13      2      0]
 [    33      1      1      1     15    339     35     17      0]
 [  4349     64      4      0      2     11  94004    603      0]
 [  8520     47      3      1      1     16   3378  31579      0]
 [     0      0      0      0      0      0      4      0 120617]]


In [ ]:
train_accuracy = model_RF.score(X_train, y_train)
test_accuracy = model_RF.score(X_test, y_test)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Training Accuracy: 0.9807354668410283
Test Accuracy: 0.9047253104295865


In [ ]:
importances = model_RF.feature_importances_

for feature, importance in zip(X.columns, importances):
    print(f"{feature}: {importance}")

ID: 0.5547531749237203
CODE_GENDER: 0.00599985270770024
FLAG_OWN_CAR: 0.006279854657403384
FLAG_OWN_REALTY: 0.005753956556466663
CNT_CHILDREN: 0.0068779045164684294
AMT_INCOME_TOTAL: 0.03402901409091537
NAME_INCOME_TYPE: 0.009255482671240043
NAME_EDUCATION_TYPE: 0.007356583311360671
NAME_FAMILY_STATUS: 0.010727815040274438
NAME_HOUSING_TYPE: 0.0058250516761673
DAYS_BIRTH: 0.04643500924583207
DAYS_EMPLOYED: 0.03714414863868854
FLAG_MOBIL: 0.0
FLAG_WORK_PHONE: 0.004398094058373815
FLAG_PHONE: 0.005065406666986125
FLAG_EMAIL: 0.0033610397772630323
OCCUPATION_TYPE: 0.019593147339357964
CNT_FAM_MEMBERS: 0.009593915565819287
MONTHS_BALANCE: 0.22755054855596235
